In [ ]:
import pandas as pd
import pickle
import networkx as nx

**Manual Input**: Set the path.

In [ ]:
path='C:/Users/acoub/OneDrive/Desktop/DSDM/Thesis/ChileGov/methodology_blank'
sep='/'

In [ ]:
path_scripts = path+sep+'update_existing'
path_data= path+sep+'data'

In [ ]:
%cd {path_data}
! twarc2 network rt_for_network_upd.json --format csv network_upd.csv --edges retweet

In [ ]:
edges=pd.read_csv(path_data+sep+"network_upd.csv",header=None)
edges.head()

In [ ]:
edges.columns=['retweeted_by','original_author','weight','rts']
edges['len']=edges['weight'].str.len()

In [ ]:
# Given that the weight comes in a string format, we should extract only the digits, depending on the long of the string we know how many characters we should extract
sub_edges=[]
for lon in range(edges['len'].min(),edges['len'].max()+1):
    sub_edges.append(edges[edges.len==lon])

In [ ]:
numb=[]
for i in range(0,len(sub_edges)):
    numb.append(-i-1)
for df,j in zip(sub_edges,numb):
    weights=df['weight'].str[j:]
    df['weight']=weights
    df['weight']=df['weight'].astype(int)

In [ ]:
edges=pd.concat(sub_edges)
edges=edges.drop(['rts','len'],axis=1)

In [ ]:
edges.head()

In [ ]:
#Eliminating self retweets.
edges=edges[edges.retweeted_by!=edges.original_author]

In [ ]:
file = path_data+sep+'final_data_set.sav'
df = pickle.load(open(file, 'rb'))
authors=list(df['author.username'])
authors = list(set(authors))


file = path_data+sep+'labeled_users.sav'
labels=pickle.load(open(file, 'rb'))

filename = path_data+sep+'new_authors.sav'
new_authors=pickle.load(open(filename, 'rb'))

In [ ]:
authors_with_label=pd.DataFrame()
authors_with_label['author.username']=new_authors
authors_with_label=authors_with_label.merge(labels[['author.username','Label']],on='author.username',how='left')
authors_with_label['Label']=authors_with_label['Label'].fillna("Unlabeled")
authors_with_label['Label'].value_counts()

### Updating the network

In [ ]:
filename = path_data+sep+'complete_network.sav'
total_G=pickle.load(open(filename, 'rb'))

left_authors=list(authors_with_label[authors_with_label.Label=="Left"]['author.username'])
right_authors=list(authors_with_label[authors_with_label.Label=="Right"]['author.username'])
unlabeled_authors=list(authors_with_label[authors_with_label.Label=="Unlabeled"]['author.username'])

In [ ]:
total_G.add_nodes_from(left_authors, af="Left")
total_G.add_nodes_from(right_authors, af="Right")
total_G.add_nodes_from(unlabeled_authors, af="Unlabeled")

In [ ]:
for i in edges.index:
    if edges['retweeted_by'][i] in authors and edges['original_author'][i] in authors:
        total_G.add_edge(edges['retweeted_by'][i],edges['original_author'][i],weight=edges['weight'][i])

In [ ]:
#Export the updated network
filename = path_data+sep+'complete_network.sav'
pickle.dump((total_G), open(filename, 'wb'))